# Pickling cuML Models for Persistence

This notebook demonstrates simple pickling of both single-GPU and multi-GPU cuML models for persistence

In [1]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

## Single GPU Model Pickling

All single-GPU estimators are pickleable. The following example demonstrates the creation of a synthetic dataset, training, and pickling of the resulting model for storage. Trained single-GPU models can also be used to distribute the inference on a Dask cluster, which the `Distributed Model Pickling` section below demonstrates.

In [2]:
from cuml.datasets import make_blobs

X, y = make_blobs(n_samples=50,
                  n_features=10,
                  centers=5,
                  cluster_std=0.4,
                  random_state=0)

In [3]:
from cuml.cluster import KMeans

model = KMeans(n_clusters=5)

model.fit(X)

KMeans()

In [4]:
import pickle

pickle.dump(model, open("kmeans_model.pkl", "wb"))

In [5]:
model = pickle.load(open("kmeans_model.pkl", "rb"))

In [6]:
model.cluster_centers_

array([[-5.768463  ,  2.3276033 , -3.7457771 , -1.8541754 , -5.1695833 ,
         7.6670885 ,  2.7118316 ,  8.495609  ,  1.7038486 ,  1.1884269 ],
       [ 4.6476874 ,  8.37788   , -9.070581  ,  9.459332  ,  8.450422  ,
        -1.0210547 ,  3.3920872 , -7.8629856 , -0.75276613,  0.48384127],
       [-2.9414442 ,  4.6401706 , -4.5027533 ,  2.2855108 ,  1.644645  ,
        -2.4937892 , -5.2241607 , -1.5499196 , -8.063638  ,  2.816936  ],
       [-4.271077  ,  5.561165  , -5.6640916 , -1.8229512 , -9.2925    ,
         0.73028314,  4.4586773 , -2.8876224 , -5.125774  ,  9.694357  ],
       [ 5.5837426 , -4.1515303 ,  4.369667  , -3.0020502 ,  3.6388965 ,
        -4.341912  , -3.3187115 ,  6.503671  , -6.865036  , -1.0266498 ]],
      dtype=float32)

## Distributed Model Pickling

The distributed estimator wrappers inside of the `cuml.dask` are not intended to be pickled directly. The Dask cuML estimators provide a function `get_combined_model()`, which returns the trained single-GPU model for pickling. The combined model can be used for inference on a single-GPU, and the `ParallelPostFit` wrapper from the [Dask-ML](https://ml.dask.org/meta-estimators.html) library can be used to perform distributed inference on a Dask cluster.

In [7]:
from dask.distributed import Client
from dask_cuda import LocalCUDACluster

cluster = LocalCUDACluster()
client = Client(cluster)
client

Client Scheduler: tcp://127.0.0.1:38594 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 1 Cores: 1 Memory: 251.80 GiB


In [8]:
from cuml.dask.datasets import make_blobs

n_workers = len(client.scheduler_info()["workers"].keys())

X, y = make_blobs(n_samples=5000, 
                  n_features=30,
                  centers=5, 
                  cluster_std=0.4, 
                  random_state=0,
                  n_parts=n_workers*5)

X = X.persist()
y = y.persist()

In [9]:
from cuml.dask.cluster import KMeans

dist_model = KMeans(n_clusters=5)

In [10]:
dist_model.fit(X)

In [11]:
import pickle

single_gpu_model = dist_model.get_combined_model()
pickle.dump(single_gpu_model, open("kmeans_model.pkl", "wb"))

In [12]:
single_gpu_model = pickle.load(open("kmeans_model.pkl", "rb"))

In [13]:
single_gpu_model.cluster_centers_

array([[ 4.809873  ,  8.42267   , -9.23902   ,  9.379145  ,  8.499882  ,
        -1.0592816 ,  3.3437853 , -7.8026123 , -0.59463334,  0.26447597,
         5.5073953 , -4.1069794 ,  4.2890763 , -2.8172047 ,  3.615015  ,
        -4.1612997 , -3.6209638 ,  6.2185297 , -6.946046  , -1.0828305 ,
        -5.8267703 ,  2.225877  , -3.8601213 , -1.6974078 , -5.313416  ,
         7.579578  ,  2.918748  ,  8.540429  ,  1.5523201 ,  1.0841806 ],
       [-2.8941858 ,  4.4741893 , -4.447567  ,  2.3820987 ,  1.7478831 ,
        -2.5046253 , -5.20833   , -1.6937687 , -8.134756  ,  2.64683   ,
        -4.3163643 ,  5.56554   , -5.7321987 , -1.7384953 , -9.344659  ,
         0.7084658 ,  4.4358397 , -2.9009    , -4.948638  ,  9.695304  ,
         8.366521  , -6.247453  , -6.3494725 ,  1.9546975 ,  4.157615  ,
        -9.167905  ,  4.607067  ,  8.788589  ,  6.8644233 ,  2.2319875 ],
       [-4.665716  , -9.558954  ,  6.6572294 ,  4.4401307 ,  2.1730306 ,
         2.5904033 ,  0.58001   ,  6.2550344 , -8